# ParFuMor (version YAML & Objets)

- Ajout d'un %store pour la gestion des numéros de kanoniks (16/04/20)

- Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [35]:
# -*- coding: utf8 -*-
import os
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
#import cellbell 

In [36]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

# Gestion partagée des numéros à traiter

le block suivant permet de partager les numéros à traiter entre les différents Notebooks.
- %store -r variable lit la variable dans le stock
- %store variable stocke la variable

In [37]:
%store -r numerosKalaba
numerosKalaba=[1,2,3,4,5]
numerosKalaba=[5]
print numerosKalaba
%store numerosKalaba

[5]
Stored 'numerosKalaba' (list)


In [38]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/"
serie=repertoire+"21-"
nomsKalabas=[serie+"K%d/"%num for num in numerosKalaba]

In [39]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [40]:
PFM.duplicateErrors=[]
for serie in nomsKalabas:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.safe_load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.safe_load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.safe_load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.safe_load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.safe_load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/21-K5/
NOM : Genre Nombre Cas
VER : Temps Pers Genre
ADJ : Genre Nombre Cas
PRO : Genre Nombre Cas
DET : Genre Nombre Cas
head stems
head stems,NOM
head stems,NOM,INAN
head stems,NOM,HUM
head stems,NOM,ANIM
head stems,VER
head stems,VER,VI
head stems,VER,VT
nourrit nourrir nourrir
excuse excuser excuse
head stems,ADJ
nouvelle nouveau nouvelle

=======ERREURS========

Formes homographes "nourrit" pour "nourrir" et "nourrir"
Formes homographes "excuse" pour "excuser" et "excuse"
Formes homographes "nouvelle" pour "nouveau" et "nouvelle"


In [41]:
getExemple(stems["PREP"])

u'pr\xe8s'

In [42]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

grand.A1
place.INAN.N1
près


In [43]:
paradigmes.getSigmas(["VER"])

[u'VER, Temps=PRS, Pers=3Sg, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Sg, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Sg, Genre=INAN',
 u'VER, Temps=PRS, Pers=3Pau, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Pau, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Pau, Genre=INAN',
 u'VER, Temps=PRS, Pers=3Pl, Genre=HUM',
 u'VER, Temps=PRS, Pers=3Pl, Genre=ANIM',
 u'VER, Temps=PRS, Pers=3Pl, Genre=INAN',
 u'VER, Temps=PST, Pers=3Sg, Genre=HUM',
 u'VER, Temps=PST, Pers=3Sg, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Sg, Genre=INAN',
 u'VER, Temps=PST, Pers=3Pau, Genre=HUM',
 u'VER, Temps=PST, Pers=3Pau, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Pau, Genre=INAN',
 u'VER, Temps=PST, Pers=3Pl, Genre=HUM',
 u'VER, Temps=PST, Pers=3Pl, Genre=ANIM',
 u'VER, Temps=PST, Pers=3Pl, Genre=INAN']

In [44]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [45]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')
regles.getRules("VER","Pers=3Pau")

[]

In [46]:
test=Lexeme("RAD","PRO","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
#test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

del Cas


Nombre,Pau,Pl,Sg
Genre,,,
ANIM,"RAD-a-p-at,RAD-a-p-er,RAD-a-p-ob,RAD-a-p-ab","RAD-a-l-ob,RAD-a-l-ab,RAD-a-l-at,RAD-a-l-er","RAD-a-s-ob,RAD-a-s-ab,RAD-a-s-at,RAD-a-s-er"
HUM,"RAD-u-p-ob,RAD-u-p-ab,RAD-u-p-at,RAD-u-p-er","RAD-u-l-at,RAD-u-l-er,RAD-u-l-ob,RAD-u-l-ab","RAD-u-s-at,RAD-u-s-er,RAD-u-s-ob,RAD-u-s-ab"
INAN,"RAD-i-p-at,RAD-i-p-er,RAD-i-p-ob,RAD-i-p-ab","RAD-i-l-ob,RAD-i-l-ab,RAD-i-l-at,RAD-i-l-er","RAD-i-s-ab,RAD-i-s-at,RAD-i-s-ob,RAD-i-s-er"


In [47]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{}

In [48]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

fuir.VT.V1 Zetak
trois.A2 poker
pendre.VT.V1 Duwif
placer.VT.V2 makuS
revenir.VT.V2 takan
remercier.VT.V2 sotuN
gargouille.ANIM.N2 gevop
nourrir.VT.V2 mutase
PRO g
rejoindre.VT.V2 riluS
feu.ANIM.N3 kobud
changer.VT.V2 tedut
nombreux.A2 nombru
reconnaître.VT.V1 Dagam
sortir.VT.V2 nufuS
grand.A1 savat
courir.VI.V2 tidag
pleurer-act.VI.V1 Sosak
mort.INAN.N3 Soniv
voler.VT.V1 Zukub
tornade.ANIM.N2 moger
disparaître.VT.V2 toZune
deux.A1 loNak
fille.HUM.N1 dobin
harmonie.INAN.N3 pozid
Clemencia.HUM.N2 klemansja
connaître.VT.V1 Degem
tourner.VT.V2 tiTig
demander.VT.V1 furun
cinq.A2 kink
méchant.A2 gepaT
noyer.VT.V1 kuZab
nuit.INAN.N1 sovuj
entrer.VT.V2 wajut
manoir.INAN.N2 jidaf
embrasser.VT.V2 pafit
voir.VT.V1 laDat
mère.HUM.N3 tarem
en oj
jeter.VT.V2 talok
vote.INAN.N2 temos
dans feD
vengeance.INAN.N3 vuTus
faire.VT.V1 bifed
interroger.VT.V1 guSes
demande.INAN.N2 tinup
nouvelle.INAN.N3 nuvel
DEM b
matin.INAN.N1 gafeg
voleuse.HUM.N2 Sures
excuser.VT.V2 sibaz
DEF k
place.INAN.N1 Sebep
balai.I

In [49]:
ding()